In [1]:
import requests
import pandas as pd
import json  
from pandas.io.json import json_normalize  
import psycopg2

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [2]:
link = 'https://api.omnivore.io/1.0/locations/'

headers = {
    'Api-Key': '3a01cc5779ef4e3784658607fc188719',
}

response = requests.get(link, headers=headers)
s = response.text
j = json.loads(s)


In [3]:
df = json_normalize(j['_embedded']['locations'])

In [4]:
location_ids = df['id'].tolist()
# location_ids

In [5]:
link_new = link+location_ids[0]+"/tickets/"

headers = {
    'Api-Key': '3a01cc5779ef4e3784658607fc188719',
}

response = requests.get(link_new, headers=headers)
s = response.text
j = json.loads(s)


In [16]:
link_new

'https://api.omnivore.io/1.0/locations/iMEy8xgT/tickets/'

In [7]:
df = json_normalize(j['_embedded']['tickets']) 
# df.columns

In [8]:
item_name = []
item_category = []
for x in df['_embedded.items']:
    x = json_normalize(x)
    if x.empty:
        item_name.append("None")
        item_category.append("None")
    else:
        item_all = x['_embedded.menu_item.name']
        cat_j = x['_embedded.menu_item._embedded.menu_categories']
        items = ', '.join(item_all.tolist())
        cat_all = []
        for q in cat_j:
            p = json_normalize(q)
            c = p['name']
            c = set(c)
            l = ', '.join(list(c))
            cat_all.append(l)
            cat_all = list(set(cat_all))
        categories = ', '.join((cat_all))

        item_name.append(items)
        item_category.append(categories)

In [9]:
new_df = pd.DataFrame()
new_df['Ticket_ID'] = df['id']
new_df['Total'] = df['totals.total']/100
new_df['Subtotal'] = df['totals.sub_total']/100
new_df['Tax'] = df['totals.tax']/100
new_df['Tips'] = df['totals.tips']/100
new_df['Opened At'] = df['opened_at']
new_df['Closed At'] = df['closed_at']
new_df['Order Type'] = df['_embedded.order_type.name']
new_df['Item Name'] = item_name
new_df['Category Name'] = item_category
new_df['Closed At'] = new_df['Closed At'].fillna("None")
# new_df.set_index("Ticket_ID", inplace = True) 


In [10]:
new_df

,Ticket_ID,Total,Subtotal,Tax,Tips,Opened At,Closed At,Order Type,Item Name,Category Name
0,147620,0.00,0.00,0.00,0.00,1593422958,1593423275,Bar,None,None
1,147619,0.00,0.00,0.00,0.00,1593422913,1593423238,Bar,None,None
2,147618,0.00,0.00,0.00,0.00,1593422853,1593423198,Bar,None,None
3,147613,142.81,123.00,19.81,31.00,1593305350,1593310955,Dine In,"Manhattan, Old Fashioned, Spin Dip, 6oz Filet, Cream Spinach, $4 Salad, $4 Salad, Sampler, Manhattan, Old Fashioned, Manhattan","Food, Apps, Cktls N-Z, Liquor, Liquor , Sides, Food, Side, Food, Grill, Cktls A-M, Liquor, Liquor"
4,147617,8.03,6.50,1.53,0.00,1593310647,1593310717,Dine In,*Vodka,"Liquor, Vodka"
5,147612,113.56,101.00,12.56,20.00,1593305318,1593310275,Dine In,"Old Fashioned, Tanqueray, Chez & And, Taste of Fred's, Chix Pasta, PN MacMurry, PN MacMurry","Liquor, Liquor , Gin, Food, Apps, Cktls N-Z, Liquor, Liquor , Pasta, Food, Smokehouse, Food, Wine, GL Red Wine, Glass Wine"
6,147616,1.93,1.75,0.18,0.00,1593310045,1593310084,Dine In,50th Cheesecake,"Dessert, Desssert, Food"
7,147559,201.23,178.75,22.48,0.00,1593292506,1593308540,Dine In,"HH Andouille Chz, BTL CAB Sebastia, Chivas , Brandy, Chivas , Chix Art, 6oz Filet, KC Steak Spec, Ribeye Spec, Brandy, Iced Tea","Scotch, Liquor, Beverages, Food, Non-alcoholic Beverage, Liquor, Cordials, Liquor , Food, Grill, Salad, Salads/Soup, Food, Bar Menu, Happy Hour, Food, Wine, Btl Red Wine, Bottled Wine"
8,147592,281.07,252.00,29.07,56.21,1593300661,1593308519,Dine In,"HH Spin Dip, Vodka Martini, Vodka Martini, Salmon Dinner, 6oz Filet, 1 lb Pork, 1/2lb Pork, Rolls/dozen, Full Rack App, Full Rack App, Full Rack App, Beans 1/2 Gallon, Slaw Quart, Slaw Quart, SAUV Matua, CAB Sebastiani","Liquor, Liquor , Martini/Speciality, Brunch Items, Catering, Food, Food, Apps, Food, Grill, Food, Seafood, Bar Menu, Happy Hour, Food, Wine, GL Red Wine, Glass Wine, Wine, GL White wine, Glass Wine"
9,147601,44.16,40.00,4.16,8.00,1593302602,1593308470,Dine In,"+HH Homewrecker, 1/2 Rack Entree, Sampler","Food, Apps, Smokehouse, Food, Draft Beer, Happy Hour, Beer"


In [12]:
hostname = '127.0.0.1'
username = 'snehgajiwala'
password = 'gajiwala'
database = 'snehgajiwala'

In [15]:
connection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
cursor = connection.cursor()



for Ticket_ID, record in new_df.groupby('Ticket_ID'):
    ticketId = str(record['Ticket_ID'].values[0])
    total = str(record['Total'].values[0])
    subtotal = str(record['Subtotal'].values[0])
    tax = str(record['Tax'].values[0])
    tips = str(record['Tips'].values[0])
    openedAt = str(record['Opened At'].values[0])
    closedAt = str(record['Closed At'].values[0])
    orderType = str(record['Order Type'].values[0])
    itemName = str(record['Item Name'].values[0])
    catName = str(record['Category Name'].values[0])


    
    postgres_insert_query =  "INSERT INTO OmnivoreDatanew VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);"
    record_to_insert = (ticketId,total,subtotal,tax,openedAt,closedAt,orderType,itemName,catName)
    cursor.execute(postgres_insert_query, record_to_insert)
    connection.commit()
    
    
    
    
    
cursor.close()
connection.close()

In [ ]:
new_df.to_csv("data_Production.csv")